In [ ]:
# Qn 1 (10 points)
# Read the file ‘grades.csv’. Write code that finds and displays the 5 student IDs 
# and their corresponding grades who received the lowest grades in the class. 
# Some students have not appeared for the exam, but they are not to be treated as 0. 
# They need to be simply skipped from the computation. (20points)

import zipfile
import os
import csv

#First unzip the file...
with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as unzipped:
    unzipped.extractall()

#We'll open the contents and read them into the buffer

with open("grades.csv", 'r') as csv_file:
    file_contents = csv_file.read()

#print(file_contents)
#If I split by comma now, I end up with a list of items that are combined but no related, and need to be split by \n still.
lines = file_contents.split("\n")
#I can't split the resulting item by ',' because 'list object has no attribute split.' I don't totally understand that yet.
#I can iterate through each element of the new list however and split it that way.

scores_only = []
for item in lines:
    fields = item.split(',')
    #When I print the second item of the fields, it's fine. When I try to append it, I get 'list index out of range.'
    #print(fields[1])
    #I'll try to append it to the output given a condition.
    if(len(item)>0):
        scores_only.append(fields[1])
    #Now we have a good clean list of scores, including blank and the descriptor.

#print(output)
#When I print the 'sorted' output, it sorts the numbers! That's handy, but they aren't in numerical order.
#I'll try to clean the output list more.

# print(sorted(scores_only))
#for item in scores_only:
    # if(type(item) == int): ##This won't work, since nothing in the list is an integer. Instead, we need to see if it's possible to convert to it.
        # score_ints_only.append(item)
#That didn't work, so let's try list comprehension.

#print(scores_only)
# score_ints_only = [item for item in scores_only if isinstance(item, int)]
#That also didn't work because there's a value error for isinstance.
#We're going to resolve this by using a 'try' block.

score_ints_only = []
for item in scores_only:
    try:
        score_ints_only.append(int(item))
    except ValueError:
        continue
#print(sorted(score_ints_only))

#Cool! Now we have a sorted list. We can just print the first few values of this.

lowest_five_values = (sorted(score_ints_only))[:5]

print(f"The lowest five scores, disregarding absent scores, is: {lowest_five_values}")


In [ ]:
# Qn 2 (20 points)
# Open and read the contents of the file – ‘2_NoofParts_assem.txt’. Perform the following (20 points):
# a)	Calculate how many entries are available in that file excluding the header.
# b)	Calculate the sum of all parts from each file. Essentially, finding the sum of all values contained in the 2nd column of the file.
# c)	Extract the part ID that has the largest associated no. of parts from the entire list.


file = "2_NoofParts_assem.txt"

import zipfile
import os
import chardet

with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as unzipped:
    unzipped.extractall()

#When I try to open here, I get a decode error for utf-8. What type of encoding is this file?

#"rb" does "read binary" so we can look deeply in the file.
with open("2_NoofParts_assem.txt", "rb") as file:
    result = chardet.detect(file.read())
    #print(result['encoding'])

#Not sure how to integrate this into the encoding = statement below. I'm sure there's a way.

with open("2_NoofParts_assem.txt", 'r', encoding = 'utf-16') as file:
    #print(file.read())
    lines = file.readlines()
    num_lines = len(lines)

print(f'There are {num_lines - 1} lines in the file, excluding the header')

#Doing my EDA here
#print(lines) 
#Based on this print statement, our values are separated by \t and followed by \t\t\n
#There's a value to 'clean' the statement but I don't remember right now.

#Let's first split the items.

split_values = []
for item in lines:
    split_values.append(item.split("\t"))

#print(split_values)

#From this print statement, we see that the header is still included. We can take it out now or later, because it will interfere with int operations.
#We can nest the entire value-stripping code block in a 'try' statement and then exit if we have a ValueError.
#We also need to account for the fact that the list now consists of lists.

values_only = []
for sublist in split_values:
    for item in sublist:
        try:
            values_only.append(int(item))
        except ValueError:
            continue
#print(values_only)

print(f"The sum of all integer values in the .txt file is {sum(values_only)}")

# c)	Extract the part ID that has the largest associated no. of parts from the entire list.

#We need to know which entry in the list of lists has the max value, then we can just navigate to it.
#My instinct is to iterate through the list increasing an index number, but I feel like there's a more elegant way.
#I learned about custom keys and will apply that here.

#This definition gives us the key value of whatever term in the master list that we want.
def extract_max_num(item):
    return (item[1])

#This item finds the sublist. The value in the brackets lets us 'skip' the header, starting at 1 and going onward.
max_part = max(split_values[1:], key=extract_max_num)

#The definition acts like a lambda function in that the value of max() is used as a parameter.

part_id = max_part[0]

print(f"The part ID with the highest # of associated parts is {part_id}")
print(f"The number of associated parts for that ID is {max_part[1]}")

In [ ]:
# Qn 3 (25 points)
# Using a python script, open the files contained in the .zip file – “3_Jobs_Completed_log.zip”. 
# Scan the files for the line that starts with the word string – “Jobs Completed.. ”. Extract the number associated with this line and for all instances that this word string appears across all log files, count the total sum across all files contained within the .zip file. (20points) 

# For example:

#  	Jobs Completed.. 10 2018-09-04 08:21:28.503153

# Extract the number 10 from this sentence which signifies the total number of jobs completed at the point in time.  
# Find for all instances in which the string - ‘Jobs Completed’ appears, 
# find the total number of jobs completed by adding the numbers from across the provided log files.

import zipfile
import chardet

filetypes = []
total_sum = 0
#Opening .zip within .zip here and extracting:
with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", 'r') as wrapper:
    with wrapper.open("Jobs_Completed_log.zip", 'r') as Jobs_Completed_file:
            with zipfile.ZipFile(Jobs_Completed_file) as Jobs_Completed_log:

#Next I'll iterate through a list of the files and detect encoding for each with chardet, since I have no idea if they are homoegenous.
#Is this overkill?
#To make the code resilient, I'll pass the encoding data to another 

                #I was doing this by habit, but it looks like it's not necessary if I work within the zipfile condition. 
                #   Not sure if this is preferable.
                # Jobs_Completed_log.extractall()

                Jobs_Completed_log_file_list = Jobs_Completed_log.namelist()
                for file in Jobs_Completed_log_file_list[:len(Jobs_Completed_log_file_list)]:
                    with Jobs_Completed_log.open(file) as f:
                        file_data = f.read()
                        encoding_result = chardet.detect(file_data)
                        encoding = encoding_result["encoding"]
                        filetypes.append(encoding)

                        #Found that a typical str.split does not work here like splitting a utf-8 file does.
                            # Resulted in a TypeError 
                            # Instead, I'll 'decode'.
                        file_data_str = file_data.decode(encoding)
                        lines = file_data_str.split("\n")

                        #Now that we have useful data, I'll search for the Jobs Completed lines, 
                        #   split then up, and grab the total number with an index.
                        for line in lines:
                            if line.startswith("Jobs Completed.. "):
                                parts = line.split(" ")
                                total_sum += int(parts[2])

#print("Filetypes:", filetypes)
print("Jobs completed:", total_sum)
#print(file_data)



In [ ]:
# Qn 4 (20 points)
# For the same files above, calculate the following:
# a)	How many jobs were completed in the period between 15th Aug 2018 to 15th Sept 2018?
# b)	The day of the year in which the maximum number of jobs was completed.
# c)	Calculate the total number of days elapsed between the time at which the first job batch was completed to the last batch completed.

import zipfile
import chardet
import datetime

#Rewriting this part to help me remember it.
filetypes = []
encoding = []
with zipfile.ZipFile("C:\Code\MFG598\Homework\Resources\HW3_Data_Files.zip", "r") as unzipped_outer:
    with unzipped_outer.open("Jobs_Completed_log.zip", "r") as inner_container:
        with zipfile.ZipFile(inner_container, "r") as unzipped_inner:

            file_list = unzipped_inner.namelist()
            for file in file_list:
                print(file)

            with unzipped_inner.open(file) as f:
            #Need to read the file into buffer so that it can be manipulated and chardet can read it
                file_data = f.read()
                encoding = chardet.detect(file_data)
                filetype = encoding_result["encoding"]
                filetypes.append(filetype)
                
                file_data_string = file_data.decode(filetype)
                lines = file_data_string.split("\n")

print(lines)



In [ ]:
# Qn 5 (25 points)
# Read the following (6_Part1.stl) triangular mesh file available in ASCII format. 
# For those of you aware of 3D printing, you will recognize this is as a .STL file. 
# For those who aren’t aware of what a .STL file is, please read up online on the format of a .STL ascii file. 
# An ASCII formatted .STL file can be opened in any text editor. 
# Every vertex of the triangle is preceeded by the word ‘vertex’ followed by the x, y and z coordinates. 
# Therefore, ONE Triangle should have 3 vertex entries. Typically, the units is not specified, 
# but you can assume it to be in millimeters. 

# Write functions that calculate the following: (20 points)
# a.	Find the total number of triangles listed in the file.
# b.	Store the coordinates of each triangle in a 3-tuple list containing N indices, where N is the total number of triangles.
# c.	List the area of each triangle in the file. Compute the total surface area of all triangles listed in the file. Read up online on the formula to calculate the area of a triangle, given three vertices.
# '''